In [2]:
# import libraries and modules in jupyter notebook environment
from datetime import datetime
from datetime import date
import pandas as pd #pd is linked to module: pandas
import math
import numpy as np #np is linked to module: numpy
from tabulate import tabulate
import corefunctions as co
import bondmodule630 as bd
import alm_module as alm
import fxmodule as fx

# if cannot find tabulate, please add this at the beginning of the chunk: pip install tabulate

# Loading Excel File and Data Cleaning

In [3]:
# use !pwd to print the current working directory

In [4]:
# read the termsheets excel file and store the values under the name xls
xls = pd.ExcelFile('/Users/stephxtchan/ECO421/A2/termsheets.xlsx')

In [5]:
# Define session date as 2022-03-18
session_date = date(2022, 3, 18)

In [6]:
# read the xls file and import the BS1, BS2, BS3 spreadsheet, convert Asset and Liability columns to string
bs_1_df = pd.read_excel(xls, sheet_name = 'BS1', converters = {'Asset:': str, 'Liability:': str})
bs_2_df = pd.read_excel(xls, sheet_name = 'BS2', converters = {'Asset:': str, 'Liability:': str})
bs_3_df = pd.read_excel(xls, sheet_name = 'BS3', converters = {'Asset:': str, 'Liability:': str})

/Users/stephxtchan/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/stephxtchan/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/stephxtchan/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:300: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [7]:
# read the xls file and import the bondmkt spreadsheet and store the values under the name bondmkt_df
bondmkt_df = pd.read_excel(xls, sheet_name = 'bondmkt')

# Load Assets and Liabilities in BS1, BS2 and BS3

In [8]:
# create two new dictionaries that stores the value of Asset and Liability for BS1
asset_1 = {}
liability_1 = {}

i = 0
for id in bs_1_df['Asset:']:
    if str(id) != 'nan':
        asset_1[str(id)] = bs_1_df['Units'][i]
        i+=1
    
i = 0
for id in bs_1_df['Liability:']:
    if str(id) != 'nan':
        liability_1[str(id)] = bs_1_df['Units.1'][i]
        i+=1

In [9]:
# create two new dictionaries that stores the value of Asset and Liability for BS2
asset_2 = {}
liability_2 = {}

i = 0
for id in bs_2_df['Asset:']:
    if str(id) != 'nan':
        asset_2[str(id)] = bs_2_df['Units'][i]
        i+=1
    
i = 0
for id in bs_2_df['Liability:']:
    if str(id) != 'nan':
        liability_2[str(id)] = bs_2_df['Units.1'][i]
        i+=1

In [10]:
# create two new dictionaries that stores the value of Asset and Liability for BS3
asset_3 = {}
liability_3 = {}

i = 0
for id in bs_3_df['Asset:']:
    if str(id) != 'nan':
        asset_3[str(id)] = bs_3_df['Units'][i]
        i+=1
    
i = 0
for id in bs_3_df['Liability:']:
    if str(id) != 'nan':
        liability_3[str(id)] = bs_3_df['Units.1'][i]
        i+=1

# Loading the k-th Scenerio


In [12]:
# Read the Scenerio spreadsheet
scen_df = pd.read_excel(xls, sheet_name='Scenario')

max_row = scen_df.shape[0]
kth_scen_dict = {}
row = int(input(f'Input n-th row of scenarios: from 1=base to last row ={max_row}: '))
for key in scen_df.columns:
    kth_scen_dict[key] = scen_df[key][row]

Input n-th row of scenarios: from 1=base to last row =77: 2


# Loading simulation time "t_sim"

In [13]:
# Read the Data spreadsheet
data_df =  pd.read_excel(xls, sheet_name = 'Data')

# export tsim from the Data spreadsheet
max_row=len(data_df)
t_sim = []
for row in range(0, max_row):
    t_sim.append(datetime.date(datetime.strptime(str(data_df.loc[row][0]), '%Y-%m-%d %H:%M:%S')))

In [14]:
# Define n as the time period from session date to each tsim 
n = []
len_t = len(t_sim)

for i in range(0, len_t):
    n.append(t_sim[i] - session_date)

In [15]:
# print n
print(n)

[datetime.timedelta(0), datetime.timedelta(days=30), datetime.timedelta(days=60), datetime.timedelta(days=90), datetime.timedelta(days=120), datetime.timedelta(days=150), datetime.timedelta(days=180), datetime.timedelta(days=210), datetime.timedelta(days=240), datetime.timedelta(days=270)]


In [16]:
# restore n as a list contain only integer values
# Create a user input for the time term
n = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270]

# Loading the Currency List for Each Bonds 

In [17]:
# Create a list of currencies used for each bonds in sheet bondmkt
currency_list = np.array(bondmkt_df['CURR_DESC'])
currency_list = str(np.unique(currency_list))

In [18]:
# Create a disctionary that links currencies used for each bonds with bond id
from_curr = {}
i = 0
for id in bondmkt_df['ISSUE_ID']:
    if str(id) != 'nan':
            from_curr[str(id)] = bondmkt_df['CURR_DESC'][i]
            i += 1

# Q1. Find Duration Gap of BS1, BS2 and BS3

## Duration Gap of Balance Sheet 1

### Set the Base Currency

In [27]:
# Set to_curr as the base currency used in BS1
# Define the terms used in fxmodule
to_curr_1 = "USD"
t1 = session_date
csv_df = pd.read_csv('/Users/stephxtchan/ECO421/A2/SpotFXFile.csv')
fx_obj = fx.FXClass(csv_df)

### Calculate the Duration Gap for Each tsim

In [28]:
duration_gap = []
#since there are some major errors to run corefunction for tsim = 2022-09-14
#Professor Yang states that we can exclude 2022-09-14 and the tsim beyond this date in our calculation
for i in range(0, 6): 
    
    # Create a dictionary that links each bonds' asset values with bond id for bonds in BS1
    asset_value_1 = {}
    for id in asset_1:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = asset_1[id]
        # find the foreign exchange rate for each bonds' currency to the base currency, which is USD in BS1
        fx_rate = fx_obj.spot_fx(from_curr[id], to_curr_1)
        bond_obj.state_procedure()
        
        # find each asset values in USD
        asset_value_1[id]= bond_obj.theo_value_scen(n[i], scen_df, row) * fx_rate
        
        # calculate the total market value of assets
        sum_asset_value_1 = sum(asset_value_1.values())
        
    # Create a dictionary that links each bonds' liability values with bond id for bonds in BS1
    liability_value_1 = {}
    for id in liability_1:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = liability_1[id]
        bond_obj.state_procedure()
        
        # find the foreign exchange rate for each bonds' currency to the base currency, which is USD in BS1
        fx_rate = fx_obj.spot_fx(from_curr[id], to_curr_1)
        
        # find each liability values in USD
        liability_value_1[id]= bond_obj.theo_value_scen(n[i], scen_df, row) * fx_rate
        
    # calculate the total market value of liabilities
    sum_liability_value_1 = sum(liability_value_1.values())
    
    # create a dictionary that stores the asset duration
    dur_asset = {}
    for id in asset_1:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = asset_1[id]
        dur_asset[id] = alm.modified_duration(co.add_terms(session_date, n[i], "days"), bond_obj, kth_scen_dict)

    # create a dictionary that stores the liability duration
    dur_liab = {}
    for id in liability_1:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = liability_1[id]
        dur_liab[id]= alm.modified_duration(co.add_terms(session_date, n[i], "days"), bond_obj, kth_scen_dict)

    # create a dictionary that stores the market value of the ith position divided by total market value of assets
    asset_weighted_duration = {}
    for id in asset_value_1:
        if str(dur_asset[id]) != 'nan':
            asset_weighted_duration[id] = dur_asset[id] * asset_value_1[id] / sum_asset_value_1
        else:
            asset_weighted_duration[id] = 0
                
    # Find the duration of total asset
    asset_duration = sum(asset_weighted_duration.values())
    
    # create a dictionary that stores the market value of the ith position divided by total market value of liabilities
    liab_weighted_duration = {}
    for id in liability_1:
        if str(dur_liab[id]) != "nan":
            liab_weighted_duration[id] = dur_liab[id] * liability_value_1[id] / sum_liability_value_1
        else:
            liab_weighted_duration[id] = 0
            
    # find the duration of total liability
    liability_duration = sum(liab_weighted_duration.values())
    
    # find The Duration Gap and store the value to duration_gap list
    duration_gap.append(asset_duration - (sum_liability_value_1/sum_asset_value_1) * liability_duration)

dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=

### Results of Duration Gap of Balance Sheet 1 at Each tsim

In [29]:
# print out the result of DGap
print('The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-03-18 is ' + str(duration_gap[0]))
print('The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-04-17 is ' + str(duration_gap[1]))
print('The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-05-17 is ' + str(duration_gap[2]))
print('The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-06-16 is ' + str(duration_gap[3]))
print('The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-07-16 is ' + str(duration_gap[4]))
print('The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-08-15 is ' + str(duration_gap[5]))

The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-03-18 is -1.5486071703342517
The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-04-17 is -1.6196062071156074
The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-05-17 is -1.941770773605029
The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-06-16 is -1.9430133262929155
The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-07-16 is -1.9552204110449947
The Duration Gap of Balance Sheet 1 (base currency: USD) at 2022-08-15 is -1.9108460689116298


In [31]:
# Create a table for the result
# Define Each cell of the rows
row_1 = ['tsim', 'Balance Sheet', 'Currency', 'Scenerio', 'Duration Gap']
row_2 = ['2022-03-18', 'BS1', 'USD', 'Base', str(duration_gap[0])]
row_3 = ['2022-04-17', 'BS1', 'USD', 'Base', str(duration_gap[1])]
row_4 = ['2022-05-17', 'BS1', 'USD', 'Base', str(duration_gap[2])]
row_5 = ['2022-06-16', 'BS1', 'USD', 'Base', str(duration_gap[3])]
row_6 = ['2022-07-16', 'BS1', 'USD', 'Base', str(duration_gap[4])]
row_7 = ['2022-08-15', 'BS1', 'USD', 'Base', str(duration_gap[5])]


# Put the rows into a table, where table here is a list of lists
table_1 = [row_1, row_2, row_3, row_4, row_5, row_6, row_7]

In [32]:
# Print the table
print(tabulate(table_1, headers='firstrow', tablefmt='fancy_grid'))

╒════════════╤═════════════════╤════════════╤════════════╤════════════════╕
│ tsim       │ Balance Sheet   │ Currency   │ Scenerio   │   Duration Gap │
╞════════════╪═════════════════╪════════════╪════════════╪════════════════╡
│ 2022-03-18 │ BS1             │ USD        │ Base       │       -1.54861 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-04-17 │ BS1             │ USD        │ Base       │       -1.61961 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-05-17 │ BS1             │ USD        │ Base       │       -1.94177 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-06-16 │ BS1             │ USD        │ Base       │       -1.94301 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-07-16 │ BS1             │ USD        │ Base       │       -1.95522 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-08-15

## Duration Gap of Balance Sheet 2

### Set the Base Currency

In [19]:
# Set to_curr as the base currency used in BS2
# Define the terms used in fxmodule
to_curr_2 = "CAD"
t1 = session_date
csv_df = pd.read_csv('/Users/stephxtchan/ECO421/A2/SpotFXFile.csv')
fx_obj = fx.FXClass(csv_df)

### Calculate the Duration Gap for Each tsim

In [20]:
duration_gap_2 = []
m = len(n)
for i in range(0, m): 
    
    # Create a dictionary that links each bonds' asset values with bond id for bonds in BS2
    asset_value_2 = {}
    for id in asset_2:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = asset_2[id]
        # find the foreign exchange rate for each bonds' currency to the base currency, which is CAD in BS2
        fx_rate = fx_obj.spot_fx(from_curr[id], to_curr_2)
        bond_obj.state_procedure()
        
        # find each asset values in CAD
        asset_value_2[id]= bond_obj.theo_value_scen(n[i], scen_df, row) * fx_rate
        
        # calculate the total market value of assets
        sum_asset_value_2 = sum(asset_value_2.values())
        
    # Create a dictionary that links each bonds' liability values with bond id for bonds in BS2
    liability_value_2 = {}
    for id in liability_2:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = liability_2[id]
        bond_obj.state_procedure()
        
        # find the foreign exchange rate for each bonds' currency to the base currency, which is CAD in BS2
        fx_rate = fx_obj.spot_fx(from_curr[id], to_curr_2)
        
        # find each liability values in CAD
        liability_value_2[id]= bond_obj.theo_value_scen(n[i], scen_df, row) * fx_rate
        
    # calculate the total market value of liabilities
    sum_liability_value_2 = sum(liability_value_2.values())
    
    # create a dictionary that stores the asset duration
    dur_asset_2 = {}
    for id in asset_2:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = asset_2[id]
        dur_asset_2[id] = alm.modified_duration(co.add_terms(session_date, n[i], "days"), bond_obj, kth_scen_dict)

    # create a dictionary that stores the liability duration
    dur_liab_2 = {}
    for id in liability_2:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = liability_2[id]
        dur_liab_2[id]= alm.modified_duration(co.add_terms(session_date, n[i], "days"), bond_obj, kth_scen_dict)

    # create a dictionary that stores the market value of the ith position divided by total market value of assets
    asset_weighted_duration_2 = {}
    for id in asset_value_2:
        if str(dur_asset_2[id]) != 'nan':
            asset_weighted_duration_2[id] = dur_asset_2[id] * asset_value_2[id] / sum_asset_value_2
        else:
            asset_weighted_duration_2[id] = 0
                
    # Find the duration of total asset
    asset_duration_2 = sum(asset_weighted_duration_2.values())
    
    # create a dictionary that stores the market value of the ith position divided by total market value of liabilities
    liab_weighted_duration_2 = {}
    for id in liability_2:
        if str(dur_liab_2[id]) != "nan":
            liab_weighted_duration_2[id] = dur_liab_2[id] * liability_value_2[id] / sum_liability_value_2
        else:
            liab_weighted_duration_2[id] = 0
            
    # find the duration of total liability
    liability_duration_2 = sum(liab_weighted_duration_2.values())
    
    # find The Duration Gap and store the value to duration_gap_2 list
    duration_gap_2.append(asset_duration_2 - (sum_liability_value_2/sum_asset_value_2) * liability_duration_2)

dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=0, exit
dfy=0 is zero, exit
bond value v=0 is <=

### Results of Duration Gap of Balance Sheet 2 at Each tsim

In [21]:
# Print out the results of duration gap of BS2
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-03-18 is ' + str(duration_gap_2[0]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-04-17 is ' + str(duration_gap_2[1]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-05-17 is ' + str(duration_gap_2[2]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-06-16 is ' + str(duration_gap_2[3]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-07-16 is ' + str(duration_gap_2[4]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-08-15 is ' + str(duration_gap_2[5]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-09-14 is ' + str(duration_gap_2[6]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-10-14 is ' + str(duration_gap_2[7]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-11-13 is ' + str(duration_gap_2[8]))
print('The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-12-13 is ' + str(duration_gap_2[9]))

The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-03-18 is 1.7630619513439925
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-04-17 is 1.8262104579591232
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-05-17 is 1.7611379729656236
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-06-16 is 1.7216756479551083
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-07-16 is 1.685055571321828
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-08-15 is 1.636839743041396
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-09-14 is 1.6267775889003662
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-10-14 is 1.5943651236365537
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-11-13 is 1.540382327914353
The Duration Gap of Balance Sheet 2 (base currency: CAD) at 2022-12-13 is 1.4943480289789925


In [23]:
# Create a table for the result
# Define Each cell of the rows
row_1 = ['tsim', 'Balance Sheet', 'Currency', 'Scenerio', 'Duration Gap']
row_2 = ['2022-03-18', 'BS2', 'CAD', 'Base', str(duration_gap_2[0])]
row_3 = ['2022-04-17', 'BS2', 'CAD', 'Base', str(duration_gap_2[1])]
row_4 = ['2022-05-17', 'BS2', 'CAD', 'Base', str(duration_gap_2[2])]
row_5 = ['2022-06-16', 'BS2', 'CAD', 'Base', str(duration_gap_2[3])]
row_6 = ['2022-07-16', 'BS2', 'CAD', 'Base', str(duration_gap_2[4])]
row_7 = ['2022-08-15', 'BS2', 'CAD', 'Base', str(duration_gap_2[5])]
row_8 = ['2022-09-14', 'BS2', 'CAD', 'Base', str(duration_gap_2[6])]
row_9 = ['2022-10-14', 'BS2', 'CAD', 'Base', str(duration_gap_2[7])]
row_10 = ['2022-11-13', 'BS2', 'CAD', 'Base', str(duration_gap_2[8])]
row_11 = ['2022-12-13', 'BS2', 'CAD', 'Base', str(duration_gap_2[9])]

# Put the rows into a table, where table here is a list of lists
table_2 = [row_1, row_2, row_3, row_4, row_5, row_6, row_7, row_8, row_9, row_10, row_11]

In [25]:
# Print the table
print(tabulate(table_2, headers='firstrow', tablefmt='fancy_grid'))

╒════════════╤═════════════════╤════════════╤════════════╤════════════════╕
│ tsim       │ Balance Sheet   │ Currency   │ Scenerio   │   Duration Gap │
╞════════════╪═════════════════╪════════════╪════════════╪════════════════╡
│ 2022-03-18 │ BS2             │ CAD        │ Base       │        1.76306 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-04-17 │ BS2             │ CAD        │ Base       │        1.82621 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-05-17 │ BS2             │ CAD        │ Base       │        1.76114 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-06-16 │ BS2             │ CAD        │ Base       │        1.72168 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-07-16 │ BS2             │ CAD        │ Base       │        1.68506 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-08-15

## Duration Gap of Balance Sheet 3

### Set the Base Currency

In [34]:
# Set to_curr as the base currency used in BS3
# Define the terms used in fxmodule
to_curr_3 = "GBP"
t1 = session_date
csv_df = pd.read_csv('/Users/stephxtchan/ECO421/A2/SpotFXFile.csv')
fx_obj = fx.FXClass(csv_df)

### Calculate the Duration Gap for Each tsim

In [35]:
duration_gap_3 = []

#since there are some major errors to run corefunction for tsim = 2022-09-14
#Professor Yang states that we can exclude 2022-09-14 and the tsim beyond this date in our calculation
for i in range(0, 6): 
    
    # Create a dictionary that links each bonds' asset values with bond id for bonds in BS3
    asset_value_3 = {}
    for id in asset_3:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = asset_3[id]
        # find the foreign exchange rate for each bonds' currency to the base currency, which is GBP in BS3
        fx_rate = fx_obj.spot_fx(from_curr[id], to_curr_3)
        bond_obj.state_procedure()
        
        # find each asset values in GBP
        asset_value_3[id]= bond_obj.theo_value_scen(n[i], scen_df, row) * fx_rate
        
        # calculate the total market value of assets
        sum_asset_value_3 = sum(asset_value_3.values())
        
    # Create a dictionary that links each bonds' liability values with bond id for bonds in BS3
    liability_value_3 = {}
    for id in liability_3:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = liability_3[id]
        bond_obj.state_procedure()
        
        # find the foreign exchange rate for each bonds' currency to the base currency, which is GBP in BS3
        fx_rate = fx_obj.spot_fx(from_curr[id], to_curr_3)
        
        # find each liability values in GBP
        liability_value_3[id]= bond_obj.theo_value_scen(n[i], scen_df, row) * fx_rate
        
    # calculate the total market value of liabilities
    sum_liability_value_3 = sum(liability_value_3.values())
    
    # create a dictionary that stores the asset duration
    dur_asset_3 = {}
    for id in asset_3:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = asset_3[id]
        dur_asset_3[id] = alm.modified_duration(co.add_terms(session_date, n[i], "days"), bond_obj, kth_scen_dict)

    # create a dictionary that stores the liability duration
    dur_liab_3 = {}
    for id in liability_3:
        bond_obj = bd.BondClass(id, 'bondmkt', xls)
        bond_obj.position_unit = liability_3[id]
        dur_liab_3[id]= alm.modified_duration(co.add_terms(session_date, n[i], "days"), bond_obj, kth_scen_dict)

    # create a dictionary that stores the market value of the ith position divided by total market value of assets
    asset_weighted_duration_3 = {}
    for id in asset_value_3:
        if str(dur_asset_3[id]) != 'nan':
            asset_weighted_duration_3[id] = dur_asset_3[id] * asset_value_3[id] / sum_asset_value_3
        else:
            asset_weighted_duration_3[id] = 0
                
    # Find the duration of total asset
    asset_duration_3 = sum(asset_weighted_duration_3.values())
    
    # create a dictionary that stores the market value of the ith position divided by total market value of liabilities
    liab_weighted_duration_3 = {}
    for id in liability_3:
        if str(dur_liab_3[id]) != "nan":
            liab_weighted_duration_3[id] = dur_liab_3[id] * liability_value_3[id] / sum_liability_value_3
        else:
            liab_weighted_duration_3[id] = 0
            
    # find the duration of total liability
    liability_duration_3 = sum(liab_weighted_duration_3.values())
    
    # find The Duration Gap and store the value to duration_gap list
    duration_gap_3.append(asset_duration_3 - (sum_liability_value_3/sum_asset_value_3) * liability_duration_3)

[3.0990605542330285]
[3.0990605542330285, 3.087634287829083]
[3.0990605542330285, 3.087634287829083, 3.0401952684467894]
[3.0990605542330285, 3.087634287829083, 3.0401952684467894, 2.989358939351792]
[3.0990605542330285, 3.087634287829083, 3.0401952684467894, 2.989358939351792, 2.9478597263299733]
[3.0990605542330285, 3.087634287829083, 3.0401952684467894, 2.989358939351792, 2.9478597263299733, 2.833578883239434]


### Results of Duration Gap of Balance Sheet 3 at Each tsim

In [37]:
# print out the result of DGap of BS3
print('The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-03-18 is ' + str(duration_gap[0]))
print('The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-04-17 is ' + str(duration_gap[1]))
print('The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-05-17 is ' + str(duration_gap[2]))
print('The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-06-16 is ' + str(duration_gap[3]))
print('The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-07-16 is ' + str(duration_gap[4]))
print('The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-08-15 is ' + str(duration_gap[5]))

The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-03-18 is -1.5486071703342517
The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-04-17 is -1.6196062071156074
The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-05-17 is -1.941770773605029
The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-06-16 is -1.9430133262929155
The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-07-16 is -1.9552204110449947
The Duration Gap of Balance Sheet 3 (base currency: GBP) at 2022-08-15 is -1.9108460689116298


In [38]:
# Create a table for the result
# Define Each cell of the rows
row_1 = ['tsim', 'Balance Sheet', 'Currency', 'Scenerio', 'Duration Gap']
row_2 = ['2022-03-18', 'BS3', 'GBP', 'Base', str(duration_gap_3[0])]
row_3 = ['2022-04-17', 'BS3', 'GBP', 'Base', str(duration_gap_3[1])]
row_4 = ['2022-05-17', 'BS3', 'GBP', 'Base', str(duration_gap_3[2])]
row_5 = ['2022-06-16', 'BS3', 'GBP', 'Base', str(duration_gap_3[3])]
row_6 = ['2022-07-16', 'BS3', 'GBP', 'Base', str(duration_gap_3[4])]
row_7 = ['2022-08-15', 'BS3', 'GBP', 'Base', str(duration_gap_3[5])]


# Put the rows into a table, where table here is a list of lists
table_3 = [row_1, row_2, row_3, row_4, row_5, row_6, row_7]

In [39]:
# Print the table
print(tabulate(table_3, headers='firstrow', tablefmt='fancy_grid'))

╒════════════╤═════════════════╤════════════╤════════════╤════════════════╕
│ tsim       │ Balance Sheet   │ Currency   │ Scenerio   │   Duration Gap │
╞════════════╪═════════════════╪════════════╪════════════╪════════════════╡
│ 2022-03-18 │ BS3             │ GBP        │ Base       │        3.09906 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-04-17 │ BS3             │ GBP        │ Base       │        3.08763 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-05-17 │ BS3             │ GBP        │ Base       │        3.0402  │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-06-16 │ BS3             │ GBP        │ Base       │        2.98936 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-07-16 │ BS3             │ GBP        │ Base       │        2.94786 │
├────────────┼─────────────────┼────────────┼────────────┼────────────────┤
│ 2022-08-15